In [ ]:
import pandas as pd
df = pd.read_csv('client_dataset.csv')
df.head()

# View the column names in the data
df.columns

# Use the describe function to gather some basic statistics
df.describe().round(2)

# Use this space to do any additional research
# and familiarize yourself with the data.
df.info()

# What three item categories had the most entries?
top_three_categories = df["category"].value_counts().nlargest(3)
print(top_three_categories)

# For the category with the most entries, which subcategory had the most entry
counts = df.groupby(['category'])[["subcategory"]].value_counts()
counts

# Which five clients had the most entries in the data?
clients = df["client_id"].value_counts().nlargest(5)
clients

# Store the client ids of those top 5 clients in a list.
clientlist = [33615, 66037, 46820, 38378, 24741]
clientlist

# How many total units (the qty column) did the client with the most entries order order?
bestclient = df[['client_id',"qty"]].set_index("client_id")
bestclient=bestclient.loc[33615]
bestclient=bestclient.groupby("client_id")["qty"].sum().reset_index()
bestclient[["client_id","qty"]]

# Create a column that calculates the subtotal for each line using the unit_price and the qty
df["line_subtotal"] = df["unit_price"] * df["qty"]
df[["unit_price","qty","line_subtotal"]].head()

# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
df["total_weight"] = df["unit_weight"]*df["qty"]
def shipping_price(row):
    if (row["total_weight"] > 50):
        return row["total_weight"] * 7
    if (row["total_weight"] <= 50):
        return row["total_weight"] * 10
df["shipping_price"] = df.apply(shipping_price, axis =1)
df[["unit_price","unit_weight","qty","total_weight","shipping_price"]].head()

# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df["line_price"] = ((df["line_subtotal"] + df["shipping_price"]) * 1.0925).round(2)
df[["line_subtotal","shipping_price","line_price"]].head()

# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df["line_cost"] = (df["unit_cost"]*df["qty"]) + df["shipping_price"]
df.head(2)

# Create a column for the profit of each line using line cost and line price
df["line_profit"]= df["line_price"] - df["line_cost"]
df.head(3)

# You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.
# Order ID 2742071 had a total price of $152,811.89
# Order ID 2173913 had a total price of $162,388.71
# Order ID 6128929 had a total price of $923,441.25

# Check your work using the totals above
confirm = df[["order_id","line_price"]]
dt= confirm.set_index("order_id")
order2173913 = dt.loc[2173913]
order2742071 = dt.loc[2742071]
order6128929 = dt.loc[6128929]
print("Order 2742071 total: $",order2742071["line_price"].sum().round(2))
print("Order 2173913 total: $",order2173913["line_price"].sum())
print("Order 6128929 total: $",order6128929["line_price"].sum())

# Use the new columns with confirmed values to find the following information.
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
spending = df[["client_id","line_price"]]
clspending=spending.set_index("client_id")
print("33615: $",clspending.loc[33615]["line_price"].sum().round(2))
print("66037: $",clspending.loc[66037]["line_price"].sum().round(2))
print("46820: $",clspending.loc[46820]["line_price"].sum().round(2))
print("38378: $",clspending.loc[38378]["line_price"].sum().round(2))
print("24741: $",clspending.loc[24741]["line_price"].sum().round(2))

# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
client=df[["client_id","qty","shipping_price","line_price","line_cost","line_profit"]].set_index("client_id")
newdata=client.loc[[33615, 66037, 46820, 38378, 24741]]
summary=pd.DataFrame(newdata.groupby("client_id")[["qty","shipping_price","line_price","line_cost","line_profit"]].sum().reset_index())
summary.head()

# Format the data and rename the columns to names suitable for presentation.
# Define the money columns. 
#money = summary["shipping_price","line_price","line_cost","line_profit"]
# Define a function that converts a dollar amount to millions.
#tried for hours, had to resort to this disgustingness
def shipping_price(row):
    if (row['shipping_price'] > 1000000):
        return row['shipping_price']/1000000
def line_price(row):
    if (row['line_price'] > 1000000):
        return row['line_price']/1000000
def beep(row):
    if (row['line_cost'] > 1000000):
        return row['line_cost']/1000000
def boop(row):
    if (row['line_profit'] > 1000000):
        return row['line_profit']/1000000
summary['shipping_price']=summary.apply(shipping_price,axis=1)
summary['line_price']=summary.apply(line_price,axis=1)
summary['line_cost']=summary.apply(beep,axis=1)
summary['line_profit']=summary.apply(boop,axis=1)
summary    


   
# Apply the currency_format_millions function to only the money columns. 

        
    

# Rename the columns to reflect the change in the money format. 

rename= summary.rename(columns={"client_id":"Client ID","qty":"Units","shipping_price":"Shipping (Millions)",\
                                "line_price":"Total Revenue (Millions)","line_cost":"Total Cost (Millions)",\
                                "line_profit":"Total Profit (Millions)"})
rename= rename.round(2)
rename


# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
sorted_final=pd.DataFrame(rename.sort_values(by="Total Profit (Millions)", ascending = False))
sorted_final.head()


